In [8]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import VarianceThreshold

import pandas as pd
import os
from sklearn import svm
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [9]:
os.chdir('C:\\Users\\iiroh\\Documents\\Projects\\Music Genre Classification\\features\\')
df = pd.read_csv('features_final.csv')

In [10]:
df.reset_index(inplace=True, drop=True)
y = df.pop('label').values
df1 = df[['centroid_max','centroid_std','contrast_skew','mfcc_0_max']]
x = df1.values

In [11]:
scaler = StandardScaler()
features = scaler.fit_transform(x)

from sklearn.model_selection import train_test_split

features_train, features_test, y_train, y_test = train_test_split(features, y, test_size=0.2, random_state = 42, stratify = y)

In [13]:
params = {
    "cls__C": [0.5, 1, 5, 10],
    "cls__kernel": ['rbf', 'sigmoid','poly'],
}

pipe_svm = Pipeline([
    ('scale', StandardScaler()),
    ('var_tresh', VarianceThreshold(threshold=0.1)),
    ('cls', SVC())
])

grid_svm = GridSearchCV(pipe_svm, params, scoring='accuracy', n_jobs=-1, cv=9,verbose=2)
grid_svm.fit(features_train, y_train)

Fitting 9 folds for each of 12 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed:    3.4s finished


GridSearchCV(cv=9,
             estimator=Pipeline(steps=[('scale', StandardScaler()),
                                       ('var_tresh',
                                        VarianceThreshold(threshold=0.1)),
                                       ('cls', SVC())]),
             n_jobs=-1,
             param_grid={'cls__C': [0.5, 1, 5, 10],
                         'cls__kernel': ['rbf', 'sigmoid', 'poly']},
             scoring='accuracy', verbose=2)

In [15]:
Model = grid_svm.best_estimator_
grid_svm.best_estimator_

Pipeline(steps=[('scale', StandardScaler()),
                ('var_tresh', VarianceThreshold(threshold=0.1)),
                ('cls', SVC(C=5))])

In [18]:
from sklearn.metrics import accuracy_score

preds = grid_svm.predict(features_test)
print("Best score on validation set (accuracy) = {:.4f}".format(grid_svm.best_score_))
print("Best score on test set (accuracy) = {:.4f}".format(accuracy_score(y_test, preds)))

Best score on validation set (accuracy) = 0.4893
Best score on test set (accuracy) = 0.4550
